In [1]:
import cv2
import os
import time
import mediapipe as mp

# Setup Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# User input
label = input("🏷️ Enter the label for the hand gesture: ").strip()
save_dir = os.path.join("hand_dataset", label)
os.makedirs(save_dir, exist_ok=True)

total_images = 3000
image_size = (200, 200)
count = 0

# Webcam setup
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("❌ Cannot access camera.")
    exit()

print(f"\n📸 Starting capture for '{label}' — total: {total_images} hand-only images.")

while count < total_images:
    success, frame = cap.read()
    if not success:
        print("❌ Failed to capture frame.")
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get bounding box of hand
            x_coords = [lm.x for lm in hand_landmarks.landmark]
            y_coords = [lm.y for lm in hand_landmarks.landmark]
            x_min = int(min(x_coords) * w) - 20
            x_max = int(max(x_coords) * w) + 20
            y_min = int(min(y_coords) * h) - 20
            y_max = int(max(y_coords) * h) + 20

            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(w, x_max)
            y_max = min(h, y_max)

            # Crop and resize
            hand_crop = frame[y_min:y_max, x_min:x_max]
            if hand_crop.size == 0:
                continue
            resized_hand = cv2.resize(hand_crop, image_size)

            # Save image
            filename = f"{label}_{count+1:04d}.jpg"
            filepath = os.path.join(save_dir, filename)
            cv2.imwrite(filepath, resized_hand)
            count += 1

            # Show for feedback
            cv2.putText(frame, f'{label}: {count}/{total_images}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
            cv2.imshow('Hand Capture (Just Hand)', frame)

    else:
        cv2.putText(frame, '✋ Show your hand clearly', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.imshow('Hand Capture (Just Hand)', frame)

    # Wait or break
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        print("🛑 Quit requested.")
        break

print(f"\n✅ Done! {count} hand images saved in: {save_dir}")
cap.release()
cv2.destroyAllWindows()


🏷️ Enter the label for the hand gesture:  N



📸 Starting capture for 'N' — total: 3000 hand-only images.

✅ Done! 3000 hand images saved in: hand_dataset\N
